In [1]:
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import numpy as np
import pandas as pd
import dask.array as da
#from tensorflow.python.client import timeline
import dask.dataframe as dd
import scipy.stats as st
import mirsig.util.utility as ut
import mirsig
import re

from sklearn.metrics import precision_recall_curve, precision_recall_fscore_support, f1_score
%matplotlib inline

In [2]:
# row major
# n = numvars
# nv = number of voters, algorithms
# mats = list of all matricies
def borda (mats, n, nv):
    
    for v in range (nv):
        mats[v] = mats[v].argsort(axis = None)
        
    borda_points = np.zeros((n*n))
    for i in range(n*n):
        # i is the current rank star between 0-15
        # rank pint is i/highest rank possible (n*n-1)
        
        for v in range (nv):
            
            borda_points[(mats[v])[i]] = borda_points[(mats[v])[i]] + (i/(n*n-1))
        
         
    return ((borda_points/nv).reshape(n,n))

# Test

In [3]:
# dream_df, column_names = ut.get_data("data/dream4/insilico_size100_2_multifactorial.tsv", sep ='\t')[0:2]
# dream_df, column_names = ut.get_data("PD_data_GEO.csv")[0:2]

In [4]:
dream_df = pd.read_csv("AD_data_GEO.csv", index_col=0)

In [5]:
dream_df.head()

,GSM2233534,GSM2233535,GSM2233537,GSM2233538,GSM2233539,GSM2233540,GSM2233541,GSM2233542,GSM2233544,GSM2233547,...,GSM2235488,GSM2235490,GSM2235495,GSM2235501,GSM2235503,GSM2235504,GSM2235512,GSM2235519,GSM2235521,GSM2235522
ENTREZ_GENE_ID,,,,,,,,,,,,,,,,,,,,,
10,1.894550,1.844536,1.407503,1.851952,1.725851,1.752923,1.471343,1.395326,2.219697,1.463018,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
100,2.723472,2.660155,2.472315,2.179213,2.443815,2.617917,2.749893,2.385487,2.638806,2.430330,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1000,7.866425,7.966718,7.696251,7.666251,7.580768,8.220708,7.711140,8.136351,7.732809,8.177074,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
10000,3.490881,3.225545,3.508853,3.770084,3.404882,3.562278,3.537456,4.147778,3.415148,3.687742,...,3.626186,2.990150,3.002777,2.733827,4.026714,3.502457,3.413461,3.809056,3.650891,3.679745
100009676,1.728233,1.907187,1.772074,1.624281,1.816555,1.878175,1.637512,1.710690,1.577785,1.568170,...,2.598129,2.309836,2.343526,2.677604,2.299704,2.337758,2.815082,2.590416,2.605727,2.574401


In [6]:
column_names = list(dream_df.index)
dream_df = dream_df.T

In [7]:
column_names

[10,
 100,
 1000,
 10000,
 100009676,
 10001,
 10002,
 10003,
 10004,
 100049716,
 10005,
 10006,
 10007,
 10008,
 10009,
 1001,
 10010,
 100101266,
 100101267,
 100101467,
 10011,
 100113403,
 100113407,
 100124700,
 100125288,
 100126784,
 100126791,
 100126793,
 100127972,
 100127983,
 100128079,
 100128098,
 100128108,
 100128124,
 100128164,
 100128185,
 100128191,
 100128198,
 100128239,
 100128252,
 100128281,
 100128288,
 100128292,
 100128298,
 100128325,
 100128327,
 100128496,
 100128590,
 100128640,
 100128644,
 100128653,
 100128731,
 100128750,
 100128751,
 100128782,
 100128822,
 100128893,
 100128927,
 100128946,
 100128977,
 100128998,
 100129034,
 100129129,
 100129175,
 100129195,
 100129196,
 100129198,
 100129250,
 100129271,
 100129361,
 100129380,
 100129387,
 100129406,
 100129482,
 100129503,
 100129516,
 100129528,
 100129550,
 100129597,
 100129716,
 100129722,
 100129792,
 100129794,
 100129827,
 100129845,
 100129858,
 100129935,
 100129961,
 10013,
 100130

In [8]:
# run mrnetb befor clr!!
mrnetb_res= mirsig.mrnetb_cuda(dream_df)

Mrnetb for : 17284
1017.004750s sec


In [9]:
np.save('intermidiate/ADmrnetb', mrnetb_res)

In [10]:
clr_res = mirsig.clr(dream_df.values,128,16)

In [11]:
np.save('intermidiate/ADclr_res', clr_res)

In [12]:
genie_res = mirsig.parallel_genie(dream_df)

100%

In [13]:
np.save('intermidiate/ADgenie', genie_res)

In [14]:
genie_no_dir = pd.DataFrame(np.maximum(genie_res.values, genie_res.values.T)) 

In [15]:
spearman_res = mirsig.spearman_df(dream_df)

In [16]:
pearson_res = mirsig.pearson_df(dream_df)

In [17]:
#mrnetb_res= mirsig.mrnetb_cuda()
pearson_res = pearson_res.abs()

In [18]:
spearman_res = spearman_res.abs()

In [19]:
kendall_res = mirsig.kendal_taub_cuda(dream_df)

Kendall for : 17284
259.814859s sec


In [20]:
borda_res = borda([mrnetb_res.values,
                   clr_res,
                   genie_res.values,
                   kendall_res.values,
                   spearman_res.values,
                   pearson_res.values],
                   pearson_res.shape[0], 6)
np.fill_diagonal(borda_res, 0)

In [21]:
np.save('intermidiate/ADdata', borda_res)

In [8]:
borda_res = np.load('intermidiate/ADdata.npy')

In [10]:
dS = pd.DataFrame (borda_res, columns= column_names, index= column_names )
edgeList = dS.stack().reset_index()
edgeList=edgeList[edgeList[0]>0.90]
edgeList[['level_0','level_1'] ].to_csv('AD_network_edgelist_90.tsv', sep='\t', header=False, index=False)
edgeList=edgeList[edgeList[0]>0.95]
edgeList[['level_0','level_1'] ].to_csv('AD_network_edgelist_95.tsv', sep='\t', header=False, index=False)
edgeList=edgeList[edgeList[0]>0.99]
edgeList[['level_0','level_1'] ].to_csv('AD_network_edgelist_99.tsv', sep='\t', header=False, index=False)